In [8]:
!pip install matplot lib
!pip install keras
!pip install tqdm
!pip3 install scikit-learn

In [9]:
from keras.callbacks import Callback
from tqdm import tqdm

class ProgressBar(Callback):
    def on_train_begin(self, logs=None):
        self.epochs = self.params['epochs']

    def on_epoch_begin(self, epoch, logs=None):
        print(f"Epoch {epoch+1}/{self.epochs}")
        self.progbar = tqdm(total=self.params['steps'])

    def on_batch_end(self, batch, logs=None):
        self.progbar.update(1)

    def on_epoch_end(self, epoch, logs=None):
        self.progbar.close()


In [10]:
# load train and test dataset
def load_dataset():
	# load dataset
	(trainX, trainY), (testX, testY) = cifar10.load_data()
	# one hot encode target values
	trainY = to_categorical(trainY)
	testY = to_categorical(testY)
	return trainX, trainY, testX, testY

# normalize data
def normalize_data(train, test):
	train_norm = train.astype('float32')
	test_norm = test.astype('float32')
	train_norm = train_norm / 255.0
	test_norm = test_norm / 255.0
	return train_norm, test_norm

# define cnn model
def define_model():
	model = Sequential()
	model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(32, 32, 3)))
	model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
	model.add(MaxPooling2D((2, 2)))
	model.add(Flatten())
	model.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
	model.add(Dense(10, activation='softmax'))
	# compile model
	opt = SGD(learning_rate=0.001, momentum=0.9)
	model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
	return model

# plot diagnostic learning curves
def summarize_diagnostics(history):
	# plot loss
	pyplot.subplot(211)
	pyplot.title('Cross Entropy Loss')
	pyplot.plot(history.history['loss'], color='blue', label='train')
	pyplot.plot(history.history['val_loss'], color='orange', label='test')
	# plot accuracy
	pyplot.subplot(212)
	pyplot.title('Classification Accuracy')
	pyplot.plot(history.history['accuracy'], color='blue', label='train')
	pyplot.plot(history.history['val_accuracy'], color='orange', label='test')
	# save plot to file
	filename = sys.argv[0].split('/')[-1]
	pyplot.savefig(filename + '_plot.png')
	pyplot.close()


In [ ]:
# example of loading the cifar10 dataset
from matplotlib import pyplot
from keras.datasets import cifar10
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Conv2D , MaxPooling2D , Flatten , Dropout , BatchNormalization
from keras.optimizers import SGD
from sklearn.model_selection import train_test_split
import sys
import numpy as np


trainX, trainY, testX, testY = load_dataset()
trainX, testX = normalize_data(trainX,testX)

trainX, valX, trainY, valY = train_test_split(trainX, trainY, test_size=0.2, random_state=42)

model = define_model()
history = model.fit(trainX, trainY, epochs=100, batch_size=64, validation_data=(valX, valY), verbose=0, callbacks=[ProgressBar()])
_, acc = model.evaluate(testX, testY, verbose=0)

print('> %.3f' % (acc * 100.0))
summarize_diagnostics(history)




Epoch 1/100


100%|██████████| 625/625 [00:10<00:00, 61.81it/s] 


Epoch 2/100


100%|██████████| 625/625 [00:05<00:00, 112.40it/s]


Epoch 3/100


...

100%|██████████| 625/625 [00:03<00:00, 195.43it/s]


Epoch 22/100


100%|██████████| 625/625 [00:03<00:00, 191.78it/s]


Epoch 23/100


 20%|██        | 125/625 [00:00<00:02, 245.62it/s]

In [7]:
model.save('CNN-'+str(acc)+'.keras')